# The goals / steps of this project are the following:

## Training / Calibration

* Download the simulator and take data in "Training Mode"

<img src="./images/Unity-Setting.png" style="width:304px;height:228px;" />

* Test out the functions in the Jupyter Notebook provided
* Add functions to detect obstacles and samples of interest (golden rocks)

<img src="./calibration_images/example_rock1.jpg" />

* Fill in the `process_image()` function with the appropriate image processing steps (perspective transform, color threshold etc.) to get from raw images to a map.  The `output_image` creating in this step should demonstrate that mapping pipeline works.

In [ ]:
def color_thresh_object(img, rgb_thresh=(137,255,99,255,0,100)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] >= rgb_thresh[0]) & (img[:,:,0] <= rgb_thresh[1]) \
                & (img[:,:,1] >= rgb_thresh[2]) & (img[:,:,1] <= rgb_thresh[3]) \
                & (img[:,:,2] >= rgb_thresh[4]) & (img[:,:,2] <= rgb_thresh[5])
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

# Identify pixels above the threshold
# Threshold of RGB > 160 does a nice job of identifying ground pixels and obstacle pixels
def color_thresh1(img, rgb_thresh=(160,160,160)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] > rgb_thresh[0])  \
                & (img[:,:,1] > rgb_thresh[1])  \
                & (img[:,:,2] > rgb_thresh[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # invert color selection 
    #obstacle_img = cv2.bitwise_not(color_select)
    obstacle_img = np.zeros_like(img[:,:,0])
    aux = np.invert(above_thresh)
    obstacle_img[aux] = 1
    # Return two binary images
    return color_select, obstacle_img

# Define a function to pass stored images to
# reading rover position and yaw angle from csv file
# This function will be used by moviepy to create an output video
def process_image(img):
    # Example of how to use the Databucket() object defined above
    # to print the current x, y and yaw values 
    # print(data.xpos[data.count], data.ypos[data.count], data.yaw[data.count])

    # TODO: 
    # 1) Define source and destination points for perspective transform
    dst_size = 5 
    bottom_offset = 6
    source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    destination = np.float32([[image.shape[1]/2 - dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - 2*dst_size - bottom_offset], 
                  [image.shape[1]/2 - dst_size, image.shape[0] - 2*dst_size - bottom_offset],
                  ])

    # 2) Apply perspective transform
    warped = perspect_transform(img, source, destination)
    # 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
    navigable_terrain, obstacle = color_thresh1(warped)
    rock_sample = color_thresh_object(warped)    
    # 4) Convert thresholded image pixel values to rover-centric coords
    obstacle_xpix, obstacle_ypix = rover_coords(obstacle)
    rock_sample_xpix, rock_sample_ypix = rover_coords(rock_sample)
    navigable_xpix, navigable_ypix = rover_coords(navigable_terrain)
    # 5) Convert rover-centric pixel values to world coords
    scale = 10
    worldmap = np.zeros((200, 200))
    xpos = data.xpos[data.count]
    ypos = data.ypos[data.count]
    yaw =  data.yaw[data.count]
    obstacle_x_world, obstacle_y_world = pix_to_world(obstacle_xpix, obstacle_ypix, xpos, 
                                ypos, yaw, 
                                worldmap.shape[0], scale)
    rock_x_world, rock_y_world = pix_to_world(rock_sample_xpix, rock_sample_ypix, xpos, 
                                ypos, yaw, 
                                worldmap.shape[0], scale)    
    navigable_x_world, navigable_y_world = pix_to_world(navigable_xpix, navigable_ypix, xpos, 
                                ypos, yaw, 
                                worldmap.shape[0], scale)    
    # 6) Update worldmap (to be displayed on right side of screen)
    data.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
    data.worldmap[rock_y_world, rock_x_world, 1] += 1
    data.worldmap[navigable_y_world, navigable_x_world, 2] += 1

    # 7) Make a mosaic image, below is some example code
        # First create a blank image (can be whatever shape you like)
    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1]*2, 3))
        # Next you can populate regions of the image with various output
        # Here I'm putting the original image in the upper left hand corner
    output_image[0:img.shape[0], 0:img.shape[1]] = img

        # Let's create more images to add to the mosaic, first a warped image
    warped = perspect_transform(img, source, destination)
        # Add the warped image in the upper right hand corner
    output_image[0:img.shape[0], img.shape[1]:] = warped

        # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.5, 0)
        # Flip map overlay so y-axis points upward and add to output_image 
    output_image[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)


        # Then putting some text over the image
    cv2.putText(output_image,"Populate this image with your analyses to make a video!", (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    data.count += 1 # Keep track of the index in the Databucket()
    
    return output_image

* Use `moviepy` to process the images in saved dataset with the `process_image()` function.  Include the video produced as part of the submission.

Note: Need to run Python code to get the mp4 video

In [1]:
from IPython.display import HTML
HTML("""
<video width="480" height="270" controls>
  <source src="{0}">
</video>
""".format('./output/test_mapping.mp4'))


## Autonomous Navigation / Mapping


* Fill in the `perception_step()` function within the `perception.py` script with the appropriate image processing functions to create a map and update `Rover()` data (similar to `process_image()` in the notebook). 

The code in perception_step() 

In [ ]:
# Identify pixels for Rock
# Threshold of RGB >= 160 low bound and RGB <= high bound, a nice job of identifying rock pixels only
def color_thresh_object(img, rgb_thresh=(137,255,99,255,0,100)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] >= rgb_thresh[0]) & (img[:,:,0] <= rgb_thresh[1]) \
                & (img[:,:,1] >= rgb_thresh[2]) & (img[:,:,1] <= rgb_thresh[3]) \
                & (img[:,:,2] >= rgb_thresh[4]) & (img[:,:,2] <= rgb_thresh[5])
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

# Identify pixels above the threshold
# Threshold of RGB > 160 does a nice job of identifying ground pixels and obstacle pixels
def color_thresh(img, rgb_thresh=(160,160,160)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] > rgb_thresh[0])  \
                & (img[:,:,1] > rgb_thresh[1])  \
                & (img[:,:,2] > rgb_thresh[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Invert color selection 
    obstacle_img = np.zeros_like(img[:,:,0])
#    obstacle_img = cv2.bitwise_not(color_select)
    aux = np.invert(above_thresh)
    obstacle_img[aux] = 1
    # Return two binary images
    return color_select, obstacle_img

# Define a function to apply a rotation to pixel positions
def rotate_pix(xpix, ypix, yaw):
    # TODO:
    # Convert yaw to radians
    # Apply a rotation
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))                            
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated

# Define a function to perform a translation
def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # TODO:
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated

# Apply the above functions in succession and update the Rover state accordingly
def perception_step(Rover):
    # Perform perception steps to update Rover()
    # TODO: 
    # NOTE: camera image is coming to you in Rover.img
    # 1) Define source and destination points for perspective transform
    # self.pos = None # Current position (x, y)
    dst_size = 5 
    bottom_offset = 6
    source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    image = Rover.img
    destination = np.float32([[image.shape[1]/2 - dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - 2*dst_size - bottom_offset], 
                  [image.shape[1]/2 - dst_size, image.shape[0] - 2*dst_size - bottom_offset],
                  ])
    # 2) Apply perspective transform
    warped = perspect_transform(Rover.img, source, destination)
    # 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
    navigable_terrain, obstacle = color_thresh(warped, (170,170,170))
    rock_sample = color_thresh_object(warped)
    # 4) Update Rover.vision_image (this will be displayed on left side of screen)
        # Example: Rover.vision_image[:,:,0] = obstacle color-thresholded binary image
        #          Rover.vision_image[:,:,1] = rock_sample color-thresholded binary image
        #          Rover.vision_image[:,:,2] = navigable terrain color-thresholded binary image
    Rover.vision_image[:,:,0] = obstacle
    Rover.vision_image[:,:,1] = rock_sample*255
    Rover.vision_image[:,:,2] = navigable_terrain*255
    # 5) Convert map image pixel values to rover-centric coords
    obstacle_xpix, obstacle_ypix = rover_coords(obstacle)
    rock_sample_xpix, rock_sample_ypix = rover_coords(rock_sample)
    navigable_xpix, navigable_ypix = rover_coords(navigable_terrain)
    # 6) Convert rover-centric pixel values to world coordinates
    scale = 10
    worldmap = np.zeros((200, 200))
    obstacle_x_world, obstacle_y_world = pix_to_world(obstacle_xpix, obstacle_ypix, Rover.pos[0], 
                                Rover.pos[1], Rover.yaw, 
                                worldmap.shape[0], scale)
    rock_x_world, rock_y_world = pix_to_world(rock_sample_xpix, rock_sample_ypix, Rover.pos[0], 
                                Rover.pos[1], Rover.yaw, 
                                worldmap.shape[0], scale)
    navigable_x_world, navigable_y_world = pix_to_world(navigable_xpix, navigable_ypix, Rover.pos[0], 
                                Rover.pos[1], Rover.yaw, 
                                worldmap.shape[0], scale)
    # 7) Update Rover worldmap (to be displayed on right side of screen)
        # Example: Rover.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
        #          Rover.worldmap[rock_y_world, rock_x_world, 1] += 1
        #          Rover.worldmap[navigable_y_world, navigable_x_world, 2] += 1
    Rover.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
    Rover.worldmap[rock_y_world, rock_x_world, 1] += 1
    Rover.worldmap[navigable_y_world, navigable_x_world, 2] += 1
    # 8) Convert rover-centric pixel positions to polar coordinates
    # Update Rover pixel distances and angles
        # Rover.nav_dists = rover_centric_pixel_distances
        # Rover.nav_angles = rover_centric_angles
    dist, angles = to_polar_coords(navigable_xpix, navigable_ypix)
    Rover.nav_dists = dist
    Rover.nav_angles = angles
        
    return Rover

* Fill in the `decision_step()` function within the `decision.py` script with conditional statements that take into consideration the outputs of the `perception_step()` in deciding how to issue throttle, brake and steering commands. 

The code in decision_step()

In [ ]:
        if Rover.vel < Rover.max_vel:
            # Set throttle value to Max throttle setting                  
            Rover.throttle = 1 # Rover.throttle + Rover.throttle_set if Rover.throttle < 1 else Rover.throttle
        else: # Else coast
            Rover.throttle = 0

        # Turn range is +/- 15 degrees, when stopped the next line will induce 4-wheel turning
        # Added a new variable: steer_try_no in Rover to count steering number on one direction, 
        # If one direction trying four time, change another direction to try
        if Rover.steer_try_no < 5:
            Rover.steer_try_no += 1
            Rover.steer = -15 if Rover.steer <= 0 else 15 # Could be more clever here about which way to turn
        else:
            Rover.steer_try_no = 0
            Rover.steer = 15 if Rover.steer >= 0 else -15


* Iterate on the perception and decision function until the rover does a reasonable job of navigating and mapping.  


* Launching in autonomous mode so the rover can navigate and map autonomously. 

<img src="./images/Unity-Mode.png" style="width:340px;height:228px;" />

Developing and testing environment:

    System Model Surface Pro 4
    Operating System Window 10    
    RAM 16GB    
    Language Python 3.5 with Jupyter Nootbook    
    Virtual System conda 4.3.21
    Simulator  Unity 5.6.1  
    Git Tool   TortoiseGit 2.4.0.2
    
Simulator Setting:

    Screen Resolution  1920 * 1080
    Graphics Quality   Fantastic
    Frame Per Second   15
    
    

### Results:

    Mapped    45-80%
    Fidelity  around 50%
    Rock Found  2-4
    
The folloing picture is one of the best result:

    It used 190.2 seconds
    Maped 80.2%
    Fidelity 58.7%
    Rock Found 4
    

<img src="./images/Unity-autonomous-2.png" style="width:640px;height:480px;" />

## Problem:
    1. The color in Vision Image area is not correct, need to fix.
    2. Fidelity percentage never over 70%, need to find why
    3. The Rover sometime stopped before a big obstacle object, because there is no "Back" command, the game has to be over.
    4. Observed the Rover is going to a cycle in a big navigable terrain, need to fix it

## Approach in this project

    1. Using lower and upper boundary in RGB number to filter Rock sample, I did try to use OpenCV HSV convertion to filter 
    the object, but using RGB number to filter Rock sample is more direct and help perormance
    2. Using cv2.bitwise_not function to invert a navigable_terrain image to obstacle image, this way is more efficiently 
    and effectively.
    3. Adding a new variable Steer try number in Rover class to help the Rover to try different direction after it is stuck 
    in an obstacle
    4. Modifing decision tree to set Rover.throttle = 1 instead of Rover.throttle_set(0.2) to speed up the Rover

## I will improve these if I were going to pursue this project further

    1. Add "Back" command
    2. Optimizing the navigation algorithm to do "Wall crawler" search 
    3. Add memory function to avoid revisiting previously maped areas

